# 04 Agent | 01 Assistants API

## Azure Environment

Necessary parameter are imported from [./Configuration/application.env]. Check [Create Environment](../../01_CreateEnvironment/01_Environment.ipynb) to setup the necessary demo environment.

## Step 1: Create OpenAIClient / EmbeddingClient

The `AzureOpenAIClient` from the Azure.AI.OpenAI NuGet package offers a unified interface to create various specialized clients, each designed to handle specific tasks.
In this notebook the `AssistantClient` is used to interact with the OpenAI Assistants API.

In [13]:
#r "nuget: Azure.AI.OpenAI, 2.0.0-beta.2"
#r "nuget: DotNetEnv, 2.5.0"

using DotNetEnv;
using Azure.AI.OpenAI;
using System.ClientModel;
using OpenAI.Assistants;

//configuration file is created during environment creation
static string configurationFile = @"../../Configuration/application.env";
Env.Load(configurationFile);

string oAiApiKey = Environment.GetEnvironmentVariable("WS_AOAI_APIKEY") ?? "WS_AOAI_APIKEY not found";
string oAiEndpoint = Environment.GetEnvironmentVariable("WS_AOAI_ENDPOINT") ?? "WS_AOAI_ENDPOINT not found";
string chatCompletionDeploymentName = Environment.GetEnvironmentVariable("WS_CHATCOMPLETION_DEPLOYMENTNAME") ?? "WS_CHATCOMPLETION_DEPLOYMENTNAME not found";

ApiKeyCredential apiKeyCredential = new ApiKeyCredential(oAiApiKey);
AzureOpenAIClient azureOpenAIClient = new AzureOpenAIClient(new Uri(oAiEndpoint), apiKeyCredential);

#pragma warning disable OPENAI001
AssistantClient assistantClient = azureOpenAIClient.GetAssistantClient();

Console.WriteLine($"AzureOpenAI Client created...");
Console.WriteLine($"AssistantClient created...");

Installed Packages Azure.AI.OpenAI, 2.0.0-beta.2 DotNetEnv, 2.5.0

AzureOpenAI Client created...
AssistantClient created...


## Step 2: Create Assistant

The class `AssistantCreationOptions' takes information to define the Assistant. The following properties are used to define the to be created Assistant:

- Name: User friendly name to identify the Assistant
- Instructions: Information how the Assistant should behave. This can be compared to the SystemMessage of classic OpenAI chat completions
- Tools: A list of tools the agent has access. In this sample the `CreateCodeInterpreter` tool is provided to the Assistant. This allows the Assistant to create and execute code if necessary (not used in this simplified sample).

The Assistant is created using the method `CreateAssistantAsync()`. 

In [14]:
AssistantCreationOptions assistantCreationOptions = new AssistantCreationOptions()
{
    Name = "Sport Assistant",
    Instructions = "You help answering questions related to sport events.",
    Tools = { ToolDefinition.CreateCodeInterpreter() },
};

Assistant assistant = await assistantClient.CreateAssistantAsync(chatCompletionDeploymentName, assistantCreationOptions); 

Console.WriteLine($"Assistant: {assistant.Id} created...");

Assistant: asst_rxfOtdvJ0sceRiQkSsv5EvbS created...


## Step 3: Create Thread

A Thread is a stateful object at the OpenAI Assistants API. It stores all messages provided as input as well as all completions from LLMs.

The method `CreateThreadAsync()` is used to create a new Thread and it will be initialized with the message *Who won the Super Bowl in 2024*.



In [15]:
IList<MessageContent> messageContents = new List<MessageContent>(); 
MessageContent messageContent = MessageContent.FromText("Who won the Super Bowl in 2024?");      
messageContents.Add(messageContent);

ThreadInitializationMessage threadInitializationMessage =  
    new ThreadInitializationMessage(messageContents);

ThreadCreationOptions threadCreationOptions = new ThreadCreationOptions();
threadCreationOptions.InitialMessages.Add(threadInitializationMessage);
    
AssistantThread assistantThread = await assistantClient.CreateThreadAsync(threadCreationOptions);

Console.WriteLine($"Thread: {assistantThread.Id} created...");

Thread: thread_0f8Bg6c9WVI8jTjooAGjioUc created...


## Step 4: Create Run

After creating a Thread containing messages and creating an Assistant both entities are brought together in a Run. The run enables the Assistant to act on the provided messages. The Run is an asynchronous batch job and can be started using the `CreateRunAsync()` method.

In [16]:
RunCreationOptions runCreationOptions = new RunCreationOptions()
{
    AdditionalInstructions = "Reply with 'I don't know' if you can't answer the question."
};

ClientResult<ThreadRun> threadRun = await 
    assistantClient.CreateRunAsync(assistantThread, assistant, runCreationOptions);

Console.WriteLine($"Run: {threadRun.Value.Id} created...");



Run: run_pGFqN9vsSUAFvTuzetpGRBIb created...


## Step 6: Check Status & Retrieve Message

While the Run is executed it's status and already created messages which are placed in the Thread can be retrieved:

- `GetRunAsync()` retrieves the current state of the Run
- `GetRunSteps()` retrieves the current execution steps of the Run
- `GetMessagesAsync()` retrieves all Thread messages

In [17]:
while (!(threadRun.Value.Status == RunStatus.Completed) && !(threadRun.Value.Status == RunStatus.Failed))
{
    await foreach(var runStep in assistantClient.GetRunStepsAsync(threadRun.Value.ThreadId, threadRun.Value.Id)){
        Console.WriteLine($"Step: {runStep.Id} - {runStep.Status}");
    };
    
    await Task.Delay(1000);
    threadRun = await assistantClient.GetRunAsync(threadRun.Value.ThreadId, threadRun.Value.Id);
}

if (threadRun.Value.Status == RunStatus.Completed)
{
    await foreach(ThreadMessage threadMessage in assistantClient.GetMessagesAsync(assistantThread.Id)){
        IReadOnlyList<MessageContent> messages = threadMessage.Content; 
        messages.AsEnumerable().ToList().ForEach(msg => Console.WriteLine(msg));
        Console.WriteLine("\n\n"); 
    };
}
else if (threadRun.Value.Status == RunStatus.Failed)
{
    Console.WriteLine($"{threadRun.Value.LastError.Message}");
}

Step: step_DWc9iF03OoOOmP92B7k6b5fk - completed
I don't know.



Who won the Super Bowl in 2024?





## Step 7: Create Run Streaming

The above code cell was retrieving messages from the Thread after they have been created and have been explicitly retrieved using `GetMessagesAsync()`.
The Assistants API and the corresponding SDK provides also streaming capabilities which provide messages and tokens as they have been created and added to the Thread.

The method `CreateRunStreamingAsync()` provides a `StreamingUpdate` object which can be used to retrieve tokens as the have been added to the Thread.

In [ ]:
RunCreationOptions runCreationOptions = new RunCreationOptions()
{
    AdditionalInstructions = "Reply with 'I don't know' if you can't answer the question."
};

await foreach (StreamingUpdate streamingUpdate in assistantClient.CreateRunStreamingAsync(
    assistantThread, assistant, runCreationOptions))
{
    Console.WriteLine($"Update kind: {streamingUpdate.UpdateKind}");
    string messageContentUpdate = (streamingUpdate is MessageContentUpdate) ? 
        ((MessageContentUpdate)streamingUpdate).Text : "";
    if (!String.IsNullOrEmpty(messageContentUpdate))
        Console.WriteLine($"Completion: {messageContentUpdate}");
}

## Step 7: Clean up resources

An Assistant as well as a Thread are objects stored at the OpenAI service level. Therefore if they aren't needed any longer it is recommended to delete them. The methods `DeleteAssistantAsync()` and `DeleteThreadAsync()` can be used.

In [ ]:
await assistantClient.DeleteAssistantAsync(assistant);
await assistantClient.DeleteThreadAsync(assistantThread);

Console.WriteLine($"Assistant and Thread deleted...");       